In [1]:
import requests
from bs4 import BeautifulSoup

website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')

Scraping the url is next

In [2]:
parsed_table_data = []
myTable = soup.find('table',{'class':'wikitable sortable'})
rows = myTable.findAll('tr')
for row in rows:
    children = row.findChildren(recursive=False)
    row_text = []
    for child in children:
        clean_text = child.text
        #This is to discard reference/citation links
        clean_text = clean_text.split('&#91;')[0]
        #This is to clean the header row of the sort icons
        clean_text = clean_text.split('&#160;')[-1]
        clean_text = clean_text.strip()
        row_text.append(clean_text)
    parsed_table_data.append(row_text)


After cleaning the data, let's take a look at the obtained list:

In [3]:
parsed_table_data[0:5]

[['Postcode', 'Borough', 'Neighbourhood'],
 ['M1A', 'Not assigned', 'Not assigned'],
 ['M2A', 'Not assigned', 'Not assigned'],
 ['M3A', 'North York', 'Parkwoods'],
 ['M4A', 'North York', 'Victoria Village']]

Now let's convert it to a dataframe. The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
import pandas as pd
columns_names = ['PostalCode', 'Borough', 'Neighbourhood']
df = pd.DataFrame(parsed_table_data,columns = columns_names)
df = df.drop([0])
df.head()

,PostalCode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


Let's only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
df = df[df.Borough != 'Not assigned']
df.head(10)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Not assigned
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 

In [6]:
 df.loc[df.Neighbourhood == 'Not assigned',['Borough']]['Borough']

9    Queen's Park
Name: Borough, dtype: object

In [7]:
df.loc[df.Neighbourhood == 'Not assigned',['Neighbourhood']] =  df.loc[df.Neighbourhood == 'Not assigned',['Borough']]['Borough']
df.head(10)

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights
8,M6A,North York,Lawrence Manor
9,M7A,Queen's Park,Queen's Park
11,M9A,Etobicoke,Islington Avenue
12,M1B,Scarborough,Rouge
13,M1B,Scarborough,Malvern


If more than one neighborhood can exist in one postal code area, these will be combined into one row with the neighborhoods separated with a comma.

In [8]:
df = df.groupby(['PostalCode','Borough']).agg(lambda x: ','.join(set(x))).reset_index()

In [9]:
df.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill"
2,M1E,Scarborough,"Morningside,West Hill,Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Kennedy Park,Ionview"
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile"
8,M1M,Scarborough,"Scarborough Village West,Cliffside,Cliffcrest"
9,M1N,Scarborough,"Cliffside West,Birch Cliff"


In [10]:
df.shape

(103, 3)

I COULD NOT EXPLORE THE LAT, LONG USING GEOCODER
OVER_QUERY_LIMIT was returned

In [11]:
#!conda install -c conda-forge geocoder --yes
#import geocoder # import geocoder
#latitude = []
#longitude = []
#for postal_code in df['PostalCode']:
#    # initialize your variable to None
#    lat_lng_coords = None
#    # loop until you get the coordinates
#    while(lat_lng_coords is None):
#      g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#      lat_lng_coords = g.latlng
#    latitude = lat_lng_coords[0]
#    longitude = lat_lng_coords[1]

Let's explore the lat and long using the CSV file

In [12]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data
print('Data downloaded!')

Data downloaded!


In [13]:
latlng_df = pd.read_csv('Geospatial_Coordinates.csv')
latlng_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Now let's rename the column to match the PostalCode Column in the original dataframe before we merge them

In [14]:
latlng_df.columns = ['PostalCode','Latitude','Longitude']
latlng_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In the following step, we will use pandas to merge the two tables based on the Postal Code column

In [15]:
neighborhoods = pd.merge(df,latlng_df[['PostalCode','Latitude','Longitude']],on='PostalCode')
neighborhoods.head(10)

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Kennedy Park,Ionview",43.727929,-79.262029
7,M1L,Scarborough,"Oakridge,Clairlea,Golden Mile",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West,Cliffside,Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848


In [16]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


Let's explore Toronto's neighbourhood and plot each neighbrouhood on the map

In [17]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

address = 'Toronto, ON'
geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geopy                     1.17.0                     py_0    conda-forge


In [18]:
# create map of Toronto using Toronto's latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
map_toronto



### Get Venues including Coffee Shop Ratings - Save the resutls for further post processing
Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
Get Venue ratings as well. This will help us determine the coffee shop ratings. Venues that do not have ratings will be marked as nan. 

The code was marked down and was run once only because of the limits Foursquare has on premium calls. We will re-load the results from the saved csv file.

Let's download the size of the resulting dataframe. Reason is that Foursquare will limit your API calls after you run the notebook couple of times. 
We will use the results from the saved dataset.

### Re-load the results

In [19]:
# The code was removed by Watson Studio for sharing.

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Rating
0,"Malvern,Rouge",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant,NaN
1,"Rouge Hill,Port Union,Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar,NaN
2,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place,NaN
3,"Guildwood,Morningside,West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store,NaN
4,"Guildwood,Morningside,West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa,NaN


Some of the neighborhoods did not return enough data from Foursquare, we will proceed by removing those neighbrohoods from the list.

In [20]:
toronto_venues_grouped = toronto_venues.groupby('Neighborhood').count()
toronto_venues_grouped.reset_index(inplace=True)
missing_set = list(set(neighborhoods['Neighbourhood']) - set(toronto_venues_grouped['Neighborhood']))
missing_df = neighborhoods.loc[neighborhoods['Neighbourhood'].isin(missing_set)]

missing_df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Port Union,Highland Creek,Rouge Hill",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,West Hill,Guildwood",43.763573,-79.188711
6,M1K,Scarborough,"East Birchmount Park,Kennedy Park,Ionview",43.727929,-79.262029
10,M1P,Scarborough,"Scarborough Town Centre,Wexford Heights,Dorset...",43.757410,-79.273304
11,M1R,Scarborough,"Wexford,Maryvale",43.750072,-79.295849
13,M1T,Scarborough,"Tam O'Shanter,Clarks Corners,Sullivan",43.781638,-79.304302
14,M1V,Scarborough,"L'Amoreaux East,Steeles East,Agincourt North,M...",43.815252,-79.284577
15,M1W,Scarborough,"L'Amoreaux West,Steeles West",43.799525,-79.318389
16,M1X,Scarborough,Upper Rouge,43.836125,-79.205636


In [21]:
for hood in list(missing_df['Neighbourhood']):
    indx = neighborhoods.index[neighborhoods['Neighbourhood'] == hood]
    neighborhoods.drop(indx,inplace=True)

Let's check how many coffee shops are returned for each neighborhood

In [22]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Rating
Neighborhood,,,,,,,
Agincourt,4,4,4,4,4,4,0
"Bathurst Manor,Downsview North,Wilson Heights",17,17,17,17,17,17,2
Bayview Village,4,4,4,4,4,4,1
Berczy Park,30,30,30,30,30,30,2
Business reply mail Processing Centre969 Eastern,17,17,17,17,17,17,0
"CFB Toronto,Downsview East",3,3,3,3,3,3,0
"Cabbagetown,St. James Town",30,30,30,30,30,30,4
Caledonia-Fairbanks,6,6,6,6,6,6,0
Canada Post Gateway Processing Centre,11,11,11,11,11,11,2


Let's find out how many of these coffee shops are Starbucks or Tim hortons

In [23]:
coffeeShop = toronto_venues[toronto_venues['Venue Category'] == 'Coffee Shop']
cafe = toronto_venues[toronto_venues['Venue Category'] == 'Café']

coffeeShops = pd.concat([coffeeShop,cafe])
coffeeShops.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Rating
9,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop,6.9
10,Woburn,43.770992,-79.216917,Tim Hortons,43.770827,-79.223078,Coffee Shop,6.2
24,"Kennedy Park,Ionview,East Birchmount Park",43.727929,-79.262029,Tim Hortons,43.726895,-79.266157,Coffee Shop,5.9
72,"Steeles East,Agincourt North,Milliken,L'Amorea...",43.815252,-79.284577,Lickety's,43.816136,-79.287944,Coffee Shop,NaN
79,"Steeles West,L'Amoreaux West",43.799525,-79.318389,Tim Hortons,43.798281,-79.318317,Coffee Shop,6.2


In [24]:
print('There are {} total coffee shops in the dataset.'.format(len(coffeeShops)))
coffeeShopsGrouped = coffeeShops.groupby('Venue').count()['Neighborhood']
#coffeeShopsGrouped.head()
coffeeShopsGroupedRating = coffeeShops.groupby('Venue').mean()['Venue Rating']
#coffeeShopsGrouped["Mean Rating"] = coffeeShopsGroupedRating
coffeeShopsGrouped = pd.concat([coffeeShopsGrouped, coffeeShopsGroupedRating], axis=1)
coffeeShopsGrouped.columns = ['Total',"Mean Rating"]

There are 156 total coffee shops in the dataset.


#### Let's check the total number of coffee shops 

In [25]:
coffeeShopsGrouped.sort_values(by='Total', ascending=False)

,Total,Mean Rating
Venue,,
Starbucks,24,7.450000
Tim Hortons,22,6.550000
Second Cup,5,6.320000
Aroma Espresso Bar,5,7.780000
Mos Mos Coffee,4,8.600000
Pilot Coffee Roasters,4,8.475000
Sam James Coffee Bar (SJCB),4,8.250000
Maman,3,8.500000
Balzac's Coffee,3,8.033333


#### Let's check the mean ratings of coffee shops 

In [26]:
coffeeShopsGrouped.sort_values(by='Mean Rating', ascending=False)

,Total,Mean Rating
Venue,,
Fahrenheit Coffee,1,9.300000
Boxcar Social,2,9.100000
Rooster Coffee,1,9.100000
Arvo,1,9.100000
Dineen Coffee,2,9.100000
The Moonbean Cafe,1,9.000000
Page One Cafe,1,9.000000
Contra Cafe,1,8.900000
Hailed Coffee,1,8.900000


In [27]:
# create map of Toronto's Coffee shops
map_toronto_coffee = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, name, rating in zip(coffeeShops['Venue Latitude'], coffeeShops['Venue Longitude'], coffeeShops['Venue'], coffeeShops['Venue Rating']):
    if rating is np.nan:
        rating = 'na'
    label = '{}, {}'.format(name, rating)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='brown',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto_coffee)  

map_toronto_coffee

In [28]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot.drop(['Coffee Shop', 'Café'], axis=1, inplace=True)


toronto_onehot = toronto_onehot.iloc[0:coffeeShops.shape[0],:]
toronto_onehot['CoffeeShop Name'] = coffeeShops['Venue'].values
toronto_onehot['Latitude'] = coffeeShops['Venue Latitude'].values
toronto_onehot['Longitude'] = coffeeShops['Venue Longitude'].values

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns)
for x in ['CoffeeShop Name','Latitude','Longitude']:
    fixed_columns.remove(x)
#fixed_columns.remove(x for x in ['CoffeeShop Name','Latitude','Longitude'])
fixed_columns = ['CoffeeShop Name','Latitude','Longitude'] + fixed_columns
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,CoffeeShop Name,Latitude,Longitude,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,College Arts Building,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Starbucks,43.770037,-79.221156,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Tim Hortons,43.770827,-79.223078,0,0,0,0,0,

In the following we will fill out the above dataframe by calculating the closest distance each coffee shop is from the remaining venues.
Example: 
    If we take the first Coffee Shop "Starbucks at (43.770037,79.221156), we will find the closest:
    -  Accessories store
    -  Adult Boutique
    -  Airport 
    -  etc ..
That will give an "identity" to each coffee shop depending on what venues its close to the most. 
For less complexity, we are not calculating the distance from each coffee shop to its neighbor (but we will point it out in the business analysis section)

In [29]:
#And let's examine the new dataframe size.
import math


def distance(origin, destination):
    """
    Calculate the Haversine distance.

    Parameters
    ----------
    origin : tuple of float
        (lat, long)
    destination : tuple of float
        (lat, long)

    Returns
    -------
    distance_in_km : float

    Examples
    --------
    >>> origin = (48.1372, 11.5756)  # Munich
    >>> destination = (52.5186, 13.4083)  # Berlin
    >>> round(distance(origin, destination), 1)
    504.2
    """
    lat1, lon1 = origin
    lat2, lon2 = destination
    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = (math.sin(dlat / 2) * math.sin(dlat / 2) +
         math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) *
         math.sin(dlon / 2) * math.sin(dlon / 2))
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return d
# Loop through all venue categories in Toronto's Dataframe
for venue in fixed_columns[3::]:
    venue_df = toronto_venues[toronto_venues['Venue Category'] == venue]
    col_indx = toronto_onehot.columns.get_loc(venue)
    # Loop through all Coffee shops 
    for row_indx, lat,lng in zip(range(len(toronto_onehot["Latitude"].values)),toronto_onehot["Latitude"],toronto_onehot["Longitude"]):
        distances = []
        # Loop through all occurencies of each venue category to find the closest one to the coffee shop. Example, take the distance to the closest hospital. 
        for row2_indx, lat2,lng2 in zip(range(len(venue_df["Venue Latitude"].values)),venue_df["Venue Latitude"],venue_df["Venue Longitude"]):
            d = distance((lat,lng),(lat2,lng2))
            distances.append(round(d,2))
        toronto_onehot.iloc[row_indx,col_indx] = max(distances)


In [30]:
#Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
#toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_onehot.head()

,CoffeeShop Name,Latitude,Longitude,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,College Arts Building,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Starbucks,43.770037,-79.221156,20.12,17.54,20.83,20.89,20.77,20.87,21.07,21.06,35.15,22.82,19.45,20.88,18.49,22.93,21.03,31.49,6.78,14.27,18.02,19.39,18.12,29.42,32.10,30.38,26.85,22.39,18.95,19.37,31.98,19.39,13.35,19.27,20.79,24.79,20.86,15.24,22.75,21.03,17.48,19.25,28.44,35.36,24.37,12.96,19.91,26.97,16.58,12.11,22.61,9.89,22.03,22.05,19.31,26.38,17.25,18.15,22.04,20.56,21.06,18.46,18.74,8.91,19.31,17.99,19.08,26.47,31.91,18.36,15.98,18.17,22.73,11.02,17.48,22.42,9.82,24.71,14.71,25.02,29.89,23.59,31.18,19.24,22.76,15.37,28.09,17.52,20.22,24.92

Cluster Neighborhoods
Run k-means to cluster the neighborhood into 5 clusters.


In [31]:
toronto_onehot.iloc[:,3:]

,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,College Arts Building,College Gym,College Stadium,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Empanada Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Fish Market,Flea Market,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Intersection,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Liquor Store,Lounge,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Motel,Movie Theater,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Rock Climbing Spot,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Soccer Field,Social Club,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,20.12,17.54,20.83,20.89,20.77,20.87,21.07,21.06,35.15,22.82,19.45,20.88,18.49,22.93,21.03,31.49,6.78,14.27,18.02,19.39,18.12,29.42,32.10,30.38,26.85,22.39,18.95,19.37,31.98,19.39,13.35,19.27,20.79,24.79,20.86,15.24,22.75,21.03,17.48,19.25,28.44,35.36,24.37,12.96,19.91,26.97,16.58,12.11,22.61,9.89,22.03,22.05,19.31,26.38,17.25,18.15,22.04,20.56,21.06,18.46,18.74,8.91,19.31,17.99,19.08,26.47,31.91,18.36,15.98,18.17,22.73,11.02,17.48,22.42,9.82,24.71,14.71,25.02,29.89,23.59,31.18,19.24,22.76,15.37,28.09,17.52,20.22,24.92,19.33,29.35,18.77,29.26,19.38,22.65,25.02,15.07,18.62,23.02,18.38

In [32]:
# set number of clusters
kclusters = 5

#toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_onehot.iloc[:,3:])

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 


array([3, 3, 3, 3, 3, 3, 3, 2, 3, 2], dtype=int32)

In [33]:
#Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
coffeeShops.shape
coffeeShops['Cluster Labels'] = kmeans.labels_
coffeeShops.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Venue Rating,Cluster Labels
9,Woburn,43.770992,-79.216917,Starbucks,43.770037,-79.221156,Coffee Shop,6.9,3
10,Woburn,43.770992,-79.216917,Tim Hortons,43.770827,-79.223078,Coffee Shop,6.2,3
24,"Kennedy Park,Ionview,East Birchmount Park",43.727929,-79.262029,Tim Hortons,43.726895,-79.266157,Coffee Shop,5.9,3
72,"Steeles East,Agincourt North,Milliken,L'Amorea...",43.815252,-79.284577,Lickety's,43.816136,-79.287944,Coffee Shop,NaN,3
79,"Steeles West,L'Amoreaux West",43.799525,-79.318389,Tim Hortons,43.798281,-79.318317,Coffee Shop,6.2,3


In [34]:
#Finally, let's visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(coffeeShops['Venue Latitude'], coffeeShops['Venue Longitude'], coffeeShops['Venue'], coffeeShops['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


# Examine Clusters

In [35]:
coffeeShops.groupby(['Cluster Labels']).mean()

,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Rating
Cluster Labels,,,,,
0,43.657019,-79.382725,43.657242,-79.382907,8.024211
1,43.641393,-79.552996,43.640077,-79.554072,6.128571
2,43.724386,-79.379632,43.723904,-79.378879,7.040909
3,43.770186,-79.324753,43.769357,-79.325722,6.740000
4,43.654257,-79.457811,43.654662,-79.456597,7.093750


#### Cluster 1

In [36]:
coffeeShops.loc[coffeeShops["Cluster Labels"] == 0].groupby(['Venue']).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Category,Venue Rating,Cluster Labels
Venue,,,,,,,,
Aroma Espresso Bar,1,1,1,1,1,1,1,1
Arvo,1,1,1,1,1,1,1,1
Balzac's Coffee,3,3,3,3,3,3,3,3
Boxcar Social,2,2,2,2,2,2,2,2
Cabbagetown Brew,1,1,1,1,1,1,1,1
Cafe Frappe,1,1,1,1,1,1,1,1
Cafe Plenty,1,1,1,1,1,1,1,1
Caffe Furbo,1,1,1,1,1,1,1,1
Café Plenty,1,1,1,1,1,1,1,1


#### Cluster 2

In [37]:
coffeeShops.loc[coffeeShops["Cluster Labels"] == 1].groupby(['Venue']).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Category,Venue Rating,Cluster Labels
Venue,,,,,,,,
Big Guy's Little Coffee Shop,1,1,1,1,1,1,0,1
Cafe Sympatico,1,1,1,1,1,1,1,1
Coffee Time,1,1,1,1,1,1,0,1
Lucky Dice Restaurant,1,1,1,1,1,1,1,1
Starbucks,2,2,2,2,2,2,2,2
Tim Hortons,3,3,3,3,3,3,3,3


#### Cluster 3

In [38]:
coffeeShops.loc[coffeeShops["Cluster Labels"] == 2].groupby(['Venue']).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Category,Venue Rating,Cluster Labels
Venue,,,,,,,,
Anthony's Cafe,1,1,1,1,1,1,0,1
Aroma Espresso Bar,3,3,3,3,3,3,3,3
Baretto Caffé,1,1,1,1,1,1,1,1
Country Style,1,1,1,1,1,1,0,1
Delimark Cafe,1,1,1,1,1,1,1,1
Meow Cat Cafe,1,1,1,1,1,1,1,1
Saryo,1,1,1,1,1,1,1,1
Second Cup,1,1,1,1,1,1,1,1
Starbucks,6,6,6,6,6,6,6,6


#### Cluster 4

In [39]:
coffeeShops.loc[coffeeShops["Cluster Labels"] == 3].groupby(['Venue']).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Category,Venue Rating,Cluster Labels
Venue,,,,,,,,
Aroma Espresso Bar,1,1,1,1,1,1,1,1
Lickety's,1,1,1,1,1,1,0,1
Maxim's Cafe and Patisserie,1,1,1,1,1,1,1,1
Starbucks,2,2,2,2,2,2,2,2
The Birchcliff,1,1,1,1,1,1,1,1
Tim Hortons,5,5,5,5,5,5,5,5


#### Cluster 5

In [40]:
coffeeShops.loc[coffeeShops["Cluster Labels"] == 4].groupby(['Venue']).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue Latitude,Venue Longitude,Venue Category,Venue Rating,Cluster Labels
Venue,,,,,,,,
Coffee Time,1,1,1,1,1,1,0,1
Coffee Tree Roastery,1,1,1,1,1,1,1,1
Cool Hand of a Girl,1,1,1,1,1,1,1,1
Java Joe's,1,1,1,1,1,1,1,1
Louie Craft Coffee,1,1,1,1,1,1,1,1
Reunion Island Coffee Bar,1,1,1,1,1,1,1,1
Second Cup,1,1,1,1,1,1,1,1
Starbucks,1,1,1,1,1,1,1,1
The Abbott,1,1,1,1,1,1,1,1
